In [1]:
!python -V

Python 3.9.19


In [5]:
import pickle
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [3]:
import mlflow
mlflow.set_tracking_uri(uri=" http://127.0.0.1:8080")
mlflow.set_experiment("nyc-taxi-experiment")

2024/05/21 20:54:16 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1716324856379, experiment_id='1', last_update_time=1716324856379, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [26]:
def read_dataframe(filename):
    print(f"Reading file: {filename}")
    df = pd.read_parquet(filename)
    print("File read successfully")
    
    df['lpep_dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])
    df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
    
    df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']
    df['duration'] = df['duration'].apply(lambda td: td.total_seconds() / 60)

    # Filter for trip durations between 1 and 60 minutes
    df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

    # Convert categorical columns to string type
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [27]:
#https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page
df_train = read_dataframe('/workspaces/experiment-tracking-zoomcamp/data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('/workspaces/experiment-tracking-zoomcamp/data/green_tripdata_2024-02.parquet')

Reading file: /workspaces/experiment-tracking-zoomcamp/data/green_tripdata_2024-01.parquet
File read successfully
Reading file: /workspaces/experiment-tracking-zoomcamp/data/green_tripdata_2024-02.parquet
File read successfully


In [29]:
len(df_train), len(df_val)

(54373, 51497)

In [30]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [31]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [32]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [33]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


5.9947992164797

In [40]:
with open('/workspaces/experiment-tracking-zoomcamp/models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [44]:
with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("train-data-path", "/workspaces/experiment-tracking-zoomcamp/data/green_tripdata_2024-01.parquet")
    mlflow.log_param("valid-data-path", "/workspaces/experiment-tracking-zoomcamp/data/green_tripdata_2024-02.parquet")

    alpha = 0.12
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="/workspaces/experiment-tracking-zoomcamp/models/lin_reg.bin", artifact_path="models_pickle")

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


### XGBoost 

In [45]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [48]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [49]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [50]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [22:01:05] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.24998                           
[1]	validation-rmse:6.30313                           
[2]	validation-rmse:5.83507                           
[3]	validation-rmse:5.62892                           
[4]	validation-rmse:5.52439                           
[5]	validation-rmse:5.46639                           
[6]	validation-rmse:5.43931                           
[7]	validation-rmse:5.42726                           
[8]	validation-rmse:5.41618                           
[9]	validation-rmse:5.40834                           
[10]	validation-rmse:5.40673                          
[11]	validation-rmse:5.40565                          
[12]	validation-rmse:5.40384                          
[13]	validation-rmse:5.40148                          
[14]	validation-rmse:5.40186                          
[15]	validation-rmse:5.40108                          
[16]	validation-rmse:5.39979                          
[17]	validation-rmse:5.39885                          
[18]	valid

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [22:01:22] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:6.27626                                                    
[1]	validation-rmse:5.57682                                                    
[2]	validation-rmse:5.41971                                                    
[3]	validation-rmse:5.36870                                                    
[4]	validation-rmse:5.34859                                                    
[5]	validation-rmse:5.34048                                                    
[6]	validation-rmse:5.32837                                                    
[7]	validation-rmse:5.31435                                                    
[8]	validation-rmse:5.30334                                                    
[9]	validation-rmse:5.30063                                                    
[10]	validation-rmse:5.29999                                                   
[11]	validation-rmse:5.29638                                                   
[12]	validation-rmse:5.29306            

...

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [22:38:14] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:6.77141                                                     
[1]	validation-rmse:5.80278                                                     
[2]	validation-rmse:5.44257                                                     
[3]	validation-rmse:5.32125                                                     
[4]	validation-rmse:5.27306                                                     
[5]	validation-rmse:5.25261                                                     
[6]	validation-rmse:5.24238                                                     
[7]	validation-rmse:5.23796                                                     
[8]	validation-rmse:5.23714                                                     
[9]	validation-rmse:5.23786                                                     
[10]	validation-rmse:5.23849                                                    
[11]	validation-rmse:5.23867                                                    
[12]	validation-rmse:5.23456

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [22:38:45] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.73314                                                     
[1]	validation-rmse:6.83956                                                     
[2]	validation-rmse:6.28141                                                     
[3]	validation-rmse:5.94022                                                     
[4]	validation-rmse:5.73572                                                     
[5]	validation-rmse:5.61404                                                     
[6]	validation-rmse:5.53808                                                     
[7]	validation-rmse:5.49284                                                     
[8]	validation-rmse:5.46437                                                     
[9]	validation-rmse:5.44346                                                     
[10]	validation-rmse:5.43195                                                    
[11]	validation-rmse:5.42010                                                    
[12]	validation-rmse:5.41301

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



In [53]:

best_params = {
        'learning_rate': 0.15091974224498975,
        'max_depth': 9,
        'min_child_weight': 2.30962372679798,
        'objective': 'reg:linear',
        'reg_alpha': 0.2240486133622339,
        'reg_lambda':0.024236491845265277,
        'seed': 42
        }

mlflow.xgboost.autolog()
booster = xgb.train(
            params=best_params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )

2024/05/21 22:55:14 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '431d51913ab8477985939d315409407e', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[0]	validation-rmse:8.27342


/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [22:55:15] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[1]	validation-rmse:7.61017
[2]	validation-rmse:7.09090
[3]	validation-rmse:6.68632
[4]	validation-rmse:6.37894
[5]	validation-rmse:6.14104
[6]	validation-rmse:5.96349
[7]	validation-rmse:5.82802
[8]	validation-rmse:5.72733
[9]	validation-rmse:5.65055
[10]	validation-rmse:5.59365
[11]	validation-rmse:5.54714
[12]	validation-rmse:5.51368
[13]	validation-rmse:5.48821
[14]	validation-rmse:5.46415
[15]	validation-rmse:5.44762
[16]	validation-rmse:5.43227
[17]	validation-rmse:5.41996
[18]	validation-rmse:5.41035
[19]	validation-rmse:5.40258
[20]	validation-rmse:5.39617
[21]	validation-rmse:5.39134
[22]	validation-rmse:5.38697
[23]	validation-rmse:5.38395
[24]	validation-rmse:5.38128
[25]	validation-rmse:5.37930
[26]	validation-rmse:5.37637
[27]	validation-rmse:5.37410
[28]	validation-rmse:5.37275
[29]	validation-rmse:5.37142
[30]	validation-rmse:5.36927
[31]	validation-rmse:5.36853
[32]	validation-rmse:5.36723
[33]	validation-rmse:5.36579
[34]	validation-rmse:5.36432
[35]	validation-rmse:5.

2024/05/21 22:56:01 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/05/21 22:56:01 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [22:56:01] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."
2024/05/21 22:56:04 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these